In [4]:
import folium
import os 

# Sample store data (latitude, longitude, store name)
stores = [
(37.289053,49.5967949, "کافه کتاب سایه"),
(27.2056263,56.3466863, "کال کافه"),
(30.1848969,56.9020053, "کافه بیکری دارسین"),
(35.8245018,50.9877699, "کافه دارچین") ,
(29.6562124,52.4568474, "کافه استیم"),
(35.7372385,51.3360521, "کافه رستوران فلورانس"),
(35.8066948,51.4713759, "کافه رزتو"),
(35.8411555,51.0204807, "کافه بلانچ"),
(36.4684294,52.8705307, "کافه مرو"),
(35.8095845,51.444752, "پاز کافه"),
(35.8046509,51.4767393, "روستار آوا سنتر"),
(35.8022109,51.4575356, "کافه بیکری روستار سانا سنتر"),
(35.7697793,51.4407706, "وی کافه قلهک"),
(35.7021894,51.4027318, "وی کافه فلسطین"),
(35.8117249,51.4479636, "کافه سم بی سم"),
(35.7604185,51.3419505, "نولان پونک"),
(35.7601027,51.4140428, "نولان ونک"),
(38.0593218,46.3586639, "کافه پرکو"),
(36.675851,48.4839619, "قهوه افریقا"), 
(36.6860376,48.510492, "کافه کاو"),
(34.6270093,50.8631844 ,"کافه کی آر بی"),
(34.611805,50.842724, "کافه احسان"),
(35.805677,50.9000932, "بلو باتل"),
(35.8402764,51.01513,"کافه پلاس"),
(35.803118,50.8986549,"کافه کربن"),
(32.6558321,51.6674603,"کافه دولت"),
( 29.647181,52.4913126, "کافه هاوانا"),
( 29.6661872,52.491171, "کافه گرن"),
(29.682483,52.4513018 , "کافه بیکری گنتوم"),
(28.9604402,50.8413806 , "کافه لیکارد"),
(27.1993121,56.3405275, "کافه رترو"),
(29.4606915,60.8485422, "کافه کلاغ"),
( 35.8075895,51.4464379, "کافه بیکری کروسان روشا"),
( 35.8089916,51.4400351, "کافه بیکری کروسان مژده"),
( 35.696868, 51.411270, "بوتیک هتل حنا"),
(35.703051, 51.420161, "کافه آندورا") ,
(35.750771, 51.433460, "کر کافه") , 
(35.799027, 51.502308, "ویو لانژ"), 
( 35.8166749,51.4823533, "کافه طهرون"),
(35.711208, 51.412738 , "کافه ناتور"),
(35.791253, 51.419093 , "کافه قنادی اته"), 
(35.790838, 51.420310, "کافه پرا"),
(35.809554, 51.444627, "کافه فوکوچا"),
(35.811702, 51.450277, "کافه بیکری رولن"),
( 35.697940, 51.408280, "وست پلتفرم"),
(35.752824, 51.374380, "کافه بیکری درو")
    # Add more store data as needed
]

# Define the URL of your company's logo
logo_path = os.path.expanduser("/Users/mohammad/Downloads/download.jpeg")

# Create a map centered at a locations
mymap = folium.Map(location=[32.4279, 53.6880], zoom_start=6, min_zoom=6)

# Add markers for each store with custom icons
for store in stores:
    folium.Marker(
        location=[store[0], store[1]],
        popup=store[2],
        icon=folium.CustomIcon(logo_path, icon_size=(20, 20))
    ).add_to(mymap)


# Save the map to an HTML file
mymap.save('store_map.html')


In [5]:

import folium
import requests
import random

def download_geojson(url, filename):
    response = requests.get(url)
    if response.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(response.content)
        return True
    else:
        print(f"Failed to download {url}. Status code: {response.status_code}")
        return False

def get_country_center(country_geojson):
    """
    Calculate the center coordinates of a country polygon.
    """
    country_coordinates = country_geojson['geometry']['coordinates']
    if country_geojson['geometry']['type'] == 'Polygon':
        country_coordinates = [country_coordinates]
    lats = [coord[1] for poly_coords in country_coordinates for coord in poly_coords[0]]
    lons = [coord[0] for poly_coords in country_coordinates for coord in poly_coords[0]]
    lat = sum(lats) / len(lats)
    lon = sum(lons) / len(lons)
    return [lat, lon]

def color_country_map(country_names):
    # Create a map centered around the world
    world_map = folium.Map(location=[0, 0], zoom_start=2)
    
    # Download the GeoJSON file containing country boundaries
    url = "https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/world-countries.json"
    geo_json_data = 'world-countries.json'
    if not download_geojson(url, geo_json_data):
        return None
    
    # Generate a random color for each country
    country_colors = {}
    for country_name in country_names:
        country_colors[country_name] = "#{:06x}".format(random.randint(0, 0xFFFFFF))
    
    # Color all specified countries with random colors and add flag icons
    for country_name in country_names:
        flag_icon = f"/Users/mohammad/XAV/Departemant Financial/Report Esfand Farvardin/flags/{country_name}.png"  # Assuming the flags are named after the country
        country_geojson = next((country for country in folium.features.GeoJson(geo_json_data).data['features'] if country['properties']['name'] == country_name), None)
        if country_geojson:
            center = get_country_center(country_geojson)
            folium.Marker(
                location=center,
                icon=folium.CustomIcon(flag_icon, icon_size=(20, 20)),
                popup=country_name
            ).add_to(world_map)
        
    # Color the countries
    folium.GeoJson(
        geo_json_data,
        name='geojson',
        style_function=lambda feature: {
            'fillColor': country_colors[feature['properties']['name']] if feature['properties']['name'] in country_names else 'beige',
            'color': 'black',
            'weight': 1.5,
            'fillOpacity': 0.7
        }
    ).add_to(world_map)

    # Save the map as an HTML file
    world_map.save(f"world_map_with_flags.html")
    print("Map saved as world_map_with_flags.html")

# Example usage
country_names = ["Colombia", "Panama", "Brazil","Costa Rica", "El Salvador", "Honduras", "Kenya", "Rwanda"]  # Example list of country names
color_country_map(country_names)


Map saved as world_map_with_flags.html
